# Topic Modeling using LDA

In [1]:
# # To be run only once
# if 0 == 1:
#     !pip install gensim
#     !pip install PyLDAvis
#     !pip install spacy
#     !python -m spacy download en_core_web_sm

In [2]:
import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt

import re
import spacy
import tqdm

import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.corpora as corpora

from nltk.corpus import stopwords
import nltk
import en_core_web_sm

from pprint import pprint

import pyLDAvis.gensim
import pickle 
import pyLDAvis

import time
from collections import Counter
import random

In [73]:
# Cache stop_words into hash
stop_words = stopwords.words('english')
stop_words.extend(['from'])

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

random.seed(3)

KeyboardInterrupt: 

In [74]:
pd.set_option('display.max_rows', 50000)
pd.set_option('display.max_columns', None)

In [4]:
# nltk.download('stopwords')

## Preprocessing data

In [5]:
start = time.time()

In [6]:
# one_drive_path = "C:/Users/cfowle/The Estée Lauder Companies Inc/TeamAnis - General/"
# one_drive_path = "C:/Users/asaid/The Estée Lauder Companies Inc/TeamAnis - General/"

In [7]:
# reviews = pd.read_pickle('reviews.pickle')

# cols = ['type', 'onlinepost_id', 'source_product_identifier', 'onlinestatement_id',
#        'date', 'title', 'description', 'geography', 'channel', 'product_id',
#        'rating', 'sentiment']
# reviews = reviews[cols]

# reviews = reviews[reviews['geography']=='USA']
# reviews.drop(columns=['geography'], inplace=True)

# reviews.to_pickle('reviews_filtered.pickle')

In [8]:
# reviews = pd.read_pickle('reviews_filtered.pickle')

In [9]:
# # Creating date columns in the right dtype and dropping the day of the month: 2019-02-24 => 2019-02-01
# reviews.loc[:,'clean_date'] = pd.to_datetime(reviews['date'], errors='coerce')
# if reviews['clean_date'].isna().sum() > 0:
#     print('{} rows have been dropped because the date format is wrong.'.format(reviews['clean_date'].isna().sum()))
#     display(reviews.loc[reviews['clean_date'].isna(), 'date'])
#     reviews = reviews.dropna(subset='date')
# reviews['date'] = reviews['clean_date']  
# reviews = reviews.drop('clean_date', axis=1)
# reviews['date'] = reviews['date'].dt.to_period('m')

# # Checking for missing data (NA => -1)
# if reviews['rating'].isna().sum()>0:
#     print('{} rows are missing ratings'.format(reviews['rating'].isna().sum()))
#     reviews.loc[:,'rating'] = reviews['rating'].fillna(-1).astype(int)

# if reviews['sentiment'].isna().sum()>0:
#     print('{} rows are missing sentiments'.format(reviews['sentiment'].isna().sum()))
#     reviews.loc[:,'sentiment'] = reviews['sentiment'].fillna(-1).astype(int)
    
# # Transforming rating and sentiment to dummy variables (one-hot encoding)
# reviews.loc[:,'sentiment'] = reviews['sentiment'].str.lower()
# reviews.loc[:,'rating'] = reviews['rating'].astype(int)
# reviews = pd.concat([reviews, pd.get_dummies(data=reviews[['rating','sentiment']], columns=['rating','sentiment'], dtype=int)], axis=1)

# # Readding NAs data to ratings
# reviews.loc[reviews['rating']==-1,'rating'] = np.nan
# reviews.loc[reviews['sentiment']==-1,'sentiment'] = np.nan

# # Transforming sentiment to integer data (positive:1; netural:0, negative:-1)
# reviews.loc[:,'sentiment'] = reviews['sentiment_positive'] - reviews['sentiment_negative']

# # Aggregating RR data by OnlinePost_ID

# # Creating a column to count the number of statements by review once aggreagtion happens
# reviews['nb_statements'] = reviews['sentiment']

# reviews = reviews.groupby(['type',
#                            'channel',
#                            'source_product_identifier',  
#                            'date',
#                            'onlinepost_id']).agg({
#     'description': lambda x:'. '.join(list(x)),
#     'nb_statements':'count',
#     'rating':'first',
#     'rating_1':'first',
#     'rating_2':'first',
#     'rating_3':'first',
#     'rating_4':'first',
#     'rating_5':'first',
#     'sentiment_negative':'sum',
#     'sentiment_neutral':'sum',
#     'sentiment_positive':'sum',
#     'sentiment':'mean'
# }).reset_index()

# # Normalize the one hot sentiment encoding counts (sentiment_negative, sentiment_neutral, sentiment_positive) by the nb_statement.
# reviews[['sentiment_negative', 'sentiment_neutral', 'sentiment_positive']] = reviews[['sentiment_negative', 'sentiment_neutral', 'sentiment_positive']].div(reviews['nb_statements'], axis=0)

In [10]:
# reviews.to_pickle('reviews_wrangled.pickle')

In [ ]:
# reviews = pd.read_pickle('reviews_wrangled.pickle')

In [88]:
# brands = pd.read_csv('elc_brands.csv', encoding='ISO-8859-1')
# catalogue = pd.read_csv('elc_catalogue.csv', encoding='ISO-8859-1')

# stop_words.extend(list(set(str(w).lower() for w in ' '.join(brands['ELC_Brand'].unique().tolist() +
#                                           catalogue['Major_Category'].unique().tolist() +
#                                           catalogue['Application'].unique().tolist() +
#                                           catalogue['Category'].unique().tolist() +
#                                           catalogue['SubCategory'].unique().tolist() 
#                                          ).replace('/', ' ').split())))

In [89]:
# stop_words = Counter(stop_words)

In [43]:
def preprocess(sentences, stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    for sentence in sentences:
        doc = nlp(' '.join([token for token in gensim.utils.simple_preprocess(str(sentence), deacc=True) if token not in stop_words]) )
        yield([token.lemma_ for token in doc if token.pos_ in allowed_postags])

In [ ]:
# reviews['tokens'] = list(tqdm.tqdm(preprocess(reviews['description'].values.tolist(), stop_words), position=0, leave=True, total=len(reviews)))

  3%|██▏                                                                   | 147488/4636608 [07:38<4:42:32, 264.81it/s]

In [ ]:
reviews.to_pickle('reviews_wrangled_w_tokens.pickle')

In [ ]:
reviews = reviews.read_pickle('reviews_wrangled_w_tokens.pickle')

## Using class

In [26]:
class TopicModeller():
    def __init__(self, df, column):
        self.df = df
        self.column = column
        self.docs = df[column].values.tolist()
    
    def compute_coherence_values(self, corpus, id2word, text, k, a, b):

        lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=k, 
                                               random_state=3,
                                               chunksize=100,
                                               passes=10,
                                               alpha=a,
                                               eta=b)

        coherence_model_lda = CoherenceModel(model=lda_model, texts=text, dictionary=id2word, coherence='c_v')

        return coherence_model_lda.get_coherence()
    
    def train_valid_lda(self,
                        nb_samples: int = 100000,
                        topics_range: list = [4,6,8],
                        alpha: list = [0.1, 'symmetric','asymmetric'],
                        beta: list = [0.1, 'symmetric']):
        
        nb_samples = min(nb_samples, len(self.docs))
        
        # Training set
        train = random.sample(self.docs, nb_samples)
        
        # Create Dictionary
        self.id2word = corpora.Dictionary(train)
        
        # Term Document Frequency
        self.corpus = [self.id2word.doc2bow(text) for text in train]
        
        # Result dictionary
        model_results = {'Topics': [],
                         'Alpha': [],
                         'Beta': [],
                         'Coherence': []
                        }

        # Can take a long time to run
        print('Validating hyperparameters...')
        if 1 == 1:
            pbar = tqdm.tqdm(total=(len(beta)*(len(alpha)+1)*len(topics_range)))
            # iterate through number of topics
            for k in topics_range:
                alpha.append(50/k)
                # iterate through alpha values
                for a in alpha:
                    # iterare through beta values
                    for b in beta:
                        # get the coherence score for the given parameters
                        cv = self.compute_coherence_values(corpus=self.corpus, id2word=self.id2word, text=train,
                                                      k=k, a=a, b=b)
                        # Save the model results
                        model_results['Topics'].append(k)
                        model_results['Alpha'].append(a)
                        model_results['Beta'].append(b)
                        model_results['Coherence'].append(cv)
                        pbar.update(1)
        self.cv_results = pd.DataFrame(model_results)
        self.cv_results = pd.DataFrame(model_results).sort_values("Coherence", ascending=False)
        self.cv_results.to_csv('lda_tuning_results.csv', index=False)
        pbar.close()
        
        self.best_param = self.cv_results.iloc[0]
        self.best_num_topics = int(self.best_param['Topics'])
        self.best_alpha = self.best_param['Alpha']
        self.best_beta = self.best_param['Beta']
        
        # Build LDA model
        print('Training best model...')
        self.lda_model = gensim.models.LdaMulticore(corpus=self.corpus,
                                                   id2word=self.id2word,
                                                   num_topics=self.best_num_topics,
                                                   alpha = self.best_alpha,
                                                   eta = self.best_beta,
                                                   random_state=100,
                                                   chunksize=100,
                                                   passes=10,
                                                   per_word_topics=True)
        print('Done.')
        
    def get_docs_topics(self):
        # Term Document Frequency
        corpus = [self.id2word.doc2bow(text) for text in self.docs]
        self.output = pd.concat([self.df, pd.DataFrame(gensim.matutils.corpus2csc(self.lda_model.get_document_topics(corpus)).T.toarray(), columns=['topic_'+str(i) for i in range(1,self.best_num_topics+1)])], axis=1)
        self.output.to_pickle('reviews_w_topics_test.pkl')
        return self.output

    def view(self):
        pyLDAvis.enable_notebook()
        LDAvis_prepared = pyLDAvis.gensim.prepare(self.lda_model, self.corpus, self.id2word)
        display(LDAvis_prepared)

In [23]:
good_reviews = reviews[reviews['rating']>=4]
neutral_reviews = reviews[(reviews['rating']>2) & (reviews['rating']<4)]
bad_reviews = reviews[reviews['rating']<=2]

In [25]:
good_lda.docs

[['leave', 'face', 'feel', 'silky'],
 ['daily', 'wash', 'face', 'regular', 'moisturizer'],
 ['make', 'face', 'oily', 'feel', 'silky', 'smooth', 'face', 'star'],
 ['love', 'product'],
 ['feel', 'wonderful', 'face', 'star'],
 ['quickly',
  'feel',
  'leave',
  'skin',
  'feel',
  'smooth',
  'soft',
  'wrinkle',
  'expression',
  'line',
  'attribute',
  'use',
  'product',
  'last',
  'year'],
 ['great', 'product', 'really', 'good', 'price'],
 ['great', 'product'],
 ['star'],
 ['great', 'product', 'star'],
 ['love',
  'product',
  'need',
  'much',
  'leave',
  'skin',
  'feel',
  'silky',
  'already',
  'start',
  'show',
  'improvement',
  'week'],
 ['love', 'product'],
 ['star'],
 ['line',
  'refine',
  'texture',
  'use',
  'night',
  'forehead',
  'morning',
  'makeup',
  'dramatically',
  'reduce',
  'forehead',
  'line',
  'improve',
  'skin',
  'texture'],
 ['star',
  'definately',
  'make',
  'skin',
  'feel',
  'smooth',
  'help',
  'reduce',
  'enlarged',
  'pore'],
 ['take',

In [24]:
print(len(good_reviews))
good_lda = TopicModeller(good_reviews, 'description')

  0%|                                                                                      | 0/3605467 [00:00<?, ?it/s]

3605467
Preprocessing data...


100%|█████████████████████████████████████████████████████████████████████| 3605467/3605467 [2:25:07<00:00, 414.05it/s]

  0%|                                                                                           | 0/24 [00:00<?, ?it/s]

Validating hyperparameters...



  4%|███▎                                                                          | 1/24 [33:40<12:54:38, 2020.82s/it]

KeyboardInterrupt: 

In [29]:
good_lda.docs

3605467

In [ ]:
print(len(bad_reviews))
bad_lda = TopicModeller(bad_reviews, 'description')
bad_lda.train_valid_lda()
bad_lda.view()

In [ ]:
print(len(neutral_reviews))
neutral_lda = TopicModeller(neutral_reviews, 'description')
neutral_lda.train_valid_lda()
neutral_lda.view()

## NLP Preprocessing

In [69]:
docs = pickle.load(open("docs.pickle", "rb"))

d = corpora.Dictionary(docs)

freq = pd.DataFrame(d.dfs.values(), index=d.dfs.keys(), columns=['freq'])
freq.index.name = 'idx'
freq = freq.reset_index()
freq['token'] = freq['idx'].apply(lambda x:d[x])
freq = freq.sort_values(by='freq', ascending=False)

In [75]:
freq[freq['freq']>500]

,idx,freq,token
115,114,1612289,product
8,24,1465876,use
41,44,1461919,love
114,111,1117876,great
14,20,1110963,skin
31,43,1043474,look
5,8,1008814,good
119,117,941979,color
47,45,771895,make
78,70,768706,go


In [86]:
'concealer' in stop_words

False

In [ ]:
stop_words.extend(['face', ])

In [ ]:
freq.iloc[0:50].plot.bar(x='token', y='freq', figsize=(20,20))
plt.grid()

## Validating Model

In [ ]:
# Inputs
docs = random.sample(docs, 100000)

# Create Dictionary
id2word = corpora.Dictionary(docs)
pickle.dump(id2word, open( "id2word.p", "wb" ) )

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in docs]

In [ ]:
# supporting function
def compute_coherence_values(corpus, dictionary, text, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=text, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
grid = {}
grid['Validation_Set'] = {}

nb_words = len(id2word)

# Topics range
min_topics = 6
max_topics = 10
step_size = 1
# topics_range = range(min_topics, max_topics, step_size)
topics_range = [8]
# Alpha parameter
# Added in the loop

# Alpha
alpha = [
#     0.1, 
#     'symmetric',
#     'asymmetric'
]

# Beta parameter
beta = [
    0.1, 
#     200/nb_words
]
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [
#     gensim.utils.ClippedCorpus(corpus, num_of_docs*0.05), 
#     gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
#     gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
    self.corpus
]

corpus_title = [
#     '25% Corpus',
#     '50% Corpus',
#     '75% Corpus',
    '100% Corpus'
]

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*(len(alpha)+1)*len(topics_range)))
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            alpha.append(50/k)
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, text=docs,
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    pbar.update(1)
    res = pd.DataFrame(model_results)
    res = pd.DataFrame(model_results).sort_values("Coherence", ascending=False)
    res.to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

In [ ]:
res

## Training Best Model

In [ ]:
best_param = res.iloc[0]
num_topics = best_param['Topics']
alpha = best_param['Alpha']
eta = best_param['Beta']

In [ ]:
# Build LDA model
int_start=time.time()
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       alpha = alpha,
                                       eta = eta,
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
print(time.time()-int_start)

In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

In [ ]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared

In [ ]:
lda_model.save('lda_test.model')

In [ ]:
print(time.time()-start)

## Predictions

In [ ]:
# df = pd.read_pickle('reviews_concat.pkl')

In [ ]:
# df = df.loc['2019']

In [ ]:
# print(len(df))

In [ ]:
# docs = list(tqdm.tqdm(preprocess(df.values.tolist(), stop_words), position=0, leave=True))

In [ ]:
# pickle.dump(docs, open( "docs.p", "wb" ) )

In [ ]:
docs = pickle.load(open("docs.p", "rb"))
id2word = pickle.load(open("id2word.p", "rb"))

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in docs]

# lda_model = gen?sim.models.LdaModel.load('lda.model')

output = pd.concat([df.to_frame('description').reset_index(), pd.DataFrame(gensim.matutils.corpus2csc(lda_model.get_document_topics(corpus)).T.toarray(), columns=['topic_'+str(i) for i in range(1,num_topics+1)])], axis=1, ignore_index=True)

pd.DataFrame(gensim.matutils.corpus2csc(lda_model.get_document_topics(corpus)).T.toarray(), columns=['topic_'+str(i) for i in range(1,num_topics+1)])

df.to_frame('description').reset_index()

output.to_pickle('reviews_w_topics_test.pkl')

## Appendix

In [ ]:
# # Build the bigram and trigram models
# bigram = gensim.models.Phrases(data_words, min_count=5, threshold=150) # higher threshold fewer phrases.
# # trigram = gensim.models.Phrases(bigram[data_words], threshold=150)

# # Faster way to get a sentence clubbed as a trigram/bigram
# bigram_mod = gensim.models.phrases.Phraser(bigram)
# # trigram_mod = gensim.models.phrases.Phraser(trigram)

# def remove_stopwords(texts):
#     return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# def make_bigrams(texts):
#     return [bigram_mod[doc] for doc in texts]

# def make_trigrams(texts):
#     return [trigram_mod[bigram_mod[doc]] for doc in texts]

# def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     """https://spacy.io/api/annotation"""
#     texts_out = []
#     for sent in texts:
#         doc = nlp(" ".join(sent)) 
#         texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#     return texts_out

# # Remove Stop Words
# data_words = remove_stopwords(data_words)

# # Do lemmatization keeping only noun, adj, vb, adv
# data_words = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# # Form Bigrams
# data_words = make_bigrams(data_words)

In [ ]:
# # Build LDA model
# lda_model = gensim.models.LdaMulticore(corpus=corpus,
#                                        id2word=id2word,
#                                        num_topics=8, 
#                                        random_state=100,
#                                        chunksize=100,
#                                        passes=10,
#                                        per_word_topics=True)

In [ ]:
# pprint(lda_model.print_topics())
# doc_lda = lda_model[corpus]

In [ ]:
# coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('Coherence Score: ', coherence_lda)